# Part 1: Load and clean the flight data

In [3]:
import os
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [4]:
# Get a list of dataframes for each CSV file in the data subfolder

data = []

for x in os.listdir('data1'):
    if '.csv' not in x:
        continue
    print(x)
    data.append(pd.read_csv(f'data1/{x}',encoding='unicode_escape',low_memory=False))

FileNotFoundError: [Errno 2] No such file or directory: 'data1'

In [ ]:
# Merge the list of dataframes into one, on columns
data = pd.concat(data)

In [ ]:
# Re-index the dataframe, since there are multiple rows with the same index number because it was concatenated from multiple dataframes
data.reset_index(inplace=True)

In [ ]:
# Replace date string data with actual datetime data
dates = pd.to_datetime(data.fl_date)
years = dates.dt.year
julian_dates = dates.dt.day_of_year
data['year'] = years
data['julian'] = julian_dates

In [ ]:
# Function to take an integer representing 24 hour time and convert it to a minute-of-the-day integer; front-end zero-padding needed

def hhmm_to_minutes(hhmm):
    hhmm = str(hhmm)
    hhmm = hhmm.split('.')[0]
    if len(hhmm) < 4:
        hhmm = '0' * (4-len(hhmm)) + hhmm
    hh = int(hhmm[:2]) * 60
    mm = int(hhmm[2:])
    hhmm = hh+mm
    return hhmm

In [ ]:
data_relevant = data[['year','julian','mkt_unique_carrier','mkt_carrier_fl_num','op_unique_carrier','op_carrier_fl_num','origin_airport_id','origin','dest_airport_id',
                      'dest','distance','crs_arr_time','arr_time','crs_dep_time','dep_time','crs_elapsed_time','actual_elapsed_time','dep_delay','arr_delay']]

In [ ]:
# Remove NaN values now that the NaN-heavy columns are gone
data_relevant = data_relevant.dropna()

# Convert columns with time in hhmm integers to minute-of-the-day values
for col in ['crs_dep_time','dep_time','crs_arr_time','arr_time']:
    print(col)
    data_relevant[col] = data_relevant[col].apply(hhmm_to_minutes)

# Calculate in-air delay time and the difference between CRS and actual elapsed time
data_relevant['in_air_delay'] = data_relevant.arr_delay - data_relevant.dep_delay
data_relevant['act_minus_crs'] = data_relevant.actual_elapsed_time - data_relevant.crs_elapsed_time

# Remove rows where in-air delay and actual minus CRS elapsed time differ, as these must be errors of some kind
data_relevant = data_relevant[data_relevant.in_air_delay == data_relevant.act_minus_crs]

In [ ]:
# Save the relevant, cleaned data
data_relevant.to_csv('All_Flight_Data_Cleaned.csv.gz',compression='gzip')

# Part 2: Selecting training data

In [79]:
data_relevant = pd.read_csv('All_Flight_Data_Cleaned.csv.gz',compression='gzip')

In [147]:
# Get three categories of data -- on time (within 5 minutes of scheduled arrival), 15-30 min delay, 60+ min delay
ontime = data_relevant[np.abs(data_relevant.arr_delay) <= 5]
delayed_15 = data_relevant[(data_relevant.arr_delay >= 15) & (data_relevant.arr_delay <= 30)]
delayed_60 = data_relevant[data_relevant.arr_delay >= 60]

In [148]:
# All three categories have at least one million entries, so sample one million from each:
ontime = ontime.sample(1000000)
delayed_15 = delayed_15.sample(1000000)
delayed_60 = delayed_60.sample(1000000)

In [149]:
ontime['cat_ontime'] = np.ones(len(ontime))
ontime['cat_delayed_15'] = np.zeros(len(ontime))
ontime['cat_delayed_60'] = np.zeros(len(ontime))

delayed_15['cat_ontime'] = np.zeros(len(delayed_15))
delayed_15['cat_delayed_15'] = np.ones(len(delayed_15))
delayed_15['cat_delayed_60'] = np.zeros(len(delayed_15))

delayed_60['cat_ontime'] = np.zeros(len(delayed_60))
delayed_60['cat_delayed_15'] = np.zeros(len(delayed_60))
delayed_60['cat_delayed_60'] = np.ones(len(delayed_60))

In [150]:
final_training = pd.concat([ontime,delayed_15,delayed_60])

In [151]:
final_training = final_training.sample(frac=1).reset_index(drop=True)

In [152]:
final_training.to_csv('Training_Data_Three_Categories.csv',index=False)

# Part 3: Gathering weather information for training data

In [1]:
import airportsdata as apd
import meteostat as ms
import pickle as pk
from datetime import datetime

In [2]:
final_training = pd.read_csv('Training_Data_Three_Categories.csv')

NameError: name 'pd' is not defined

In [ ]:
# Check that all IATA airport codes in the training data map to one and only one ID number, since the codes are easier to look up but the instructions say they may have changed

id_counts_by_IATA = {}
count = 0
all_codes = set(final_training.origin).union(set(final_training.dest))

for code in all_codes:
    count += 1
    print(f'\r{count} of {len(all_codes)}',end='')
    id_codes = set(final_training[final_training.origin == code].origin_airport_id)
    id_codes = id_codes.union(set(final_training[final_training.dest == code].dest_airport_id))
    id_counts_by_IATA[code] = len(id_codes)

In [ ]:
# Check that all id counts for all IATA codes are 1:

set(id_counts_by_IATA.values())

In [ ]:
# Load airport information indexed by 3-letter IATA codes
airports = apd.load('IATA')

In [ ]:
# Define a function to retrieve hourly and daily weather data for each airport, by IATA code, for 2018-01-01 to 2020-01-31

def get_weather(airport,daily=True,hourly=True,start=datetime(2018,1,1,0),end=datetime(2020,1,31,23)):
    if not (daily or hourly):
        print('Neither daily nor hourly data requested; exiting.')
        return
    airport = airport_coordinates[airport]
    airport = ms.Point(*airport)
    airport.method = 'weighted'
    airport.radius = 150000
    airport.max_count = 25
    airport.adapt_temp = False
    if daily:
        daily_data = ms.Daily(airport,start,end)
        daily_data = daily_data.normalize()
        daily_data = daily_data.interpolate(limit=10)
        daily_data = daily_data.fetch()
        daily_data = daily_data[['tavg','tmin','tmax','prcp','snow','wdir','wspd','pres']]
    if hourly:
        hourly_data = ms.Hourly(airport,start,end)
        hourly_data = hourly_data.normalize()
        hourly_data = hourly_data.interpolate(limit=48)
        hourly_data = hourly_data.fetch()
        hourly_data = hourly_data[['temp','rhum','prcp','wdir','wspd','pres']]
    if daily and hourly:
        return (daily_data, hourly_data)
    else:
        return daily_data if daily else hourly_data

In [ ]:
def get_weather(airport,daily=True,start=datetime(2018,1,1,0),end=datetime(2020,1,31,23)):
    if not (daily or hourly):
        print('Neither daily nor hourly data requested; exiting.')
        return
    airport = airport_coordinates[airport]
    airport = ms.Point(*airport)
    airport.method = 'weighted'
    airport.radius = 150000
    airport.max_count = 25
    airport.adapt_temp = False
    hourly = ms.Hourly(airport,start,end)
    hourly_data = hourly.normalize()
    hourly_data = hourly_data.interpolate(limit=48)
    hourly_data = hourly_data.fetch()
    if daily:
        daily_data = hourly.normalize()
        daily_data = daily_data.aggregate('1D')
        daily_data = daily_data.fetch()
    if daily and hourly:
        return (daily_data, hourly_data)
    else:
        return daily_data if daily else hourly_data

In [ ]:
airport_coordinates = {}

for code in all_codes:
    if code not in airports:
        while True:
            coordinates = input(f'{code} not found; enter coordinates: ')
            try:
                lat, lon = coordinates.split(', ')
                lat, lon = float(lat), float(lon)
                break
            except:
                pass
    else:
        lat, lon = airports[code]['lat'], airports[code]['lon']
    airport_coordinates[code] = (lat,lon)

In [ ]:
with open('Airport_Coordinates_Pickled_Data.pck','wb') as outfile:
    pk.dump(airport_coordinates,outfile)

In [ ]:
with open('Airport_Coordinates_Pickled_Data.pck','rb') as infile:
    airport_coordinates = pk.load(infile)

In [ ]:
ms.Hourly.clear_cache(max_age=0)
ms.Daily.clear_cache(max_age=0)
ms.Stations.clear_cache(max_age=0)

In [ ]:
d,h = get_weather('SFO')

In [ ]:
d.to_csv('TEST_SFO_AGG.csv')

In [ ]:
# Retrieve weather data for every airport IATA code, and store in dicts, indexed by IATA codes

all_codes = set(final_training.origin).union(set(final_training.dest))
all_codes = sorted(list(all_codes))

weather_hourly, weather_daily, failed_codes = {}, {}, []
count = 0

for code in all_codes:
    count += 1
    print(f'{count} of {len(all_codes)} ({code})')
    
    daily_exists = os.path.isfile(f'weather_data_daily/{code}.csv')
    hourly_exists = os.path.isfile(f'weather_data_hourly/{code}.csv')
    
    if not daily_exists:
        print(f'Getting daily for {code}')
        try:
            daily = get_weather(code,hourly=False)
            if (len(daily) > 0) and ({ sum(daily[c].isna()) for c in daily.columns } == {0}):
                print(f'\n\nSucceeded in getting new daily data for {code}')
                daily.to_csv(f'weather_data_daily/{code}.csv')
        except:
            print('Exception')
    
    if not hourly_exists:
        print(f'Getting hourly for {code}')
        try:
            hourly = get_weather(code,daily=False)
            if (len(hourly) > 0) and ({ sum(hourly[c].isna()) for c in hourly.columns } == {0}):
                print(f'\n\nSucceeded in getting new hourly data for {code}')
                hourly.to_csv(f'weather_data_hourly/{code}.csv')
        except:
            print('Exception')

# Weather data was gathered with scripts to based on the above cells to allow them to be run in parallel

In [88]:
import datetime as dt
import os
import pandas as pd
import numpy as np

In [153]:
# final_training = pd.read_csv('Training_Data_Three_Categories.csv')
final_training.loc[final_training.crs_arr_time == 1440,'crs_arr_time'] = 0
# final_training.to_csv('Training_Data_Three_Categories.csv')

In [14]:
weather_data = []

for folder in ['WeatherData01/', 'WeatherData02/', 'WeatherData03/', 'WeatherData04/', 'WeatherData05/',]:
    print(folder)
    os.chdir(folder)
    weather = {}
    for fname in [ x for x in os.listdir(os.getcwd()) if 'hourly' in x ]:
        code = fname.split('_')[0]
        data = pd.read_csv(fname,index_col='time')
        weather[code] = data
    weather_data.append(weather)
    os.chdir('..')

WeatherData01/
WeatherData02/
WeatherData03/
WeatherData04/
WeatherData05/


In [86]:
def get_arr_time(row):
    date_str = str(row.year) + str(row.julian)
    arr_date = dt.datetime.strptime(date_str, '%Y%j')
    t_dep = dt.time(row.crs_dep_time//60,row.crs_dep_time%60)
    t_arr = dt.time(row.crs_arr_time//60,row.crs_arr_time%60)
    arr_date = arr_date.replace(hour=t_arr.hour,minute=t_arr.minute)
    if t_arr < t_dep:
        arr_date += dt.timedelta(days=1)
    return arr_date
        
def get_dep_time(row):
    date_str = str(row.year) + str(row.julian)
    dep_date = dt.datetime.strptime(date_str, '%Y%j')
    t_dep = dt.time(row.crs_dep_time//60,row.crs_dep_time%60)
    dep_date = dep_date.replace(hour=t_dep.hour,minute=t_dep.minute)
    return dep_date

In [154]:
final_training = final_training.assign(arr_datetime = final_training.apply(get_arr_time,axis=1))

In [155]:
final_training = final_training.assign(dep_datetime = final_training.apply(get_dep_time,axis=1))

In [156]:
final_training = final_training.assign(arr_datetime_rounded = final_training.arr_datetime.dt.round('H'))
final_training = final_training.assign(dep_datetime_rounded = final_training.dep_datetime.dt.round('H'))

In [157]:
final_training.loc[:,['dep_temp','dep_rhum','dep_prcp','dep_wdir','dep_wspd','dep_pres']] = np.nan
final_training.loc[:,['arr_temp','arr_rhum','arr_prcp','arr_wdir','arr_wspd','arr_pres']] = np.nan

In [159]:
final_training = final_training[final_training.columns[1:]]

In [162]:
final_training.to_csv('Training_Data_Three_Categories.csv',index=False)

In [77]:
final_training = pd.read_csv('Training_Data_Three_Categories.csv')

In [66]:
def get_dep_weather_data(df):
    for i in range(len(df)):
        if ((i+1) % 1000) == 0:
            print(f'\rProcessing row {(i+1)//1000}k')
        dep_code, dep_dt = df.loc[i].origin, str(df.loc[i].dep_datetime_rounded)
        dep_weather = weather_data[2][dep_code].loc[dep_dt]
        arr_code, arr_dt = df.loc[i].dest, str(df.loc[i].arr_datetime_rounded)
        arr_weather = weather_data[2][arr_code].loc[arr_dt]
        df.loc[i,['dep_temp','dep_rhum','dep_prcp','dep_wdir','dep_wspd','dep_pres']] = list(dep_weather[['temp','rhum','prcp','wdir','wspd','pres']])
        df.loc[i,['arr_temp','arr_rhum','arr_prcp','arr_wdir','arr_wspd','arr_pres']] = list(arr_weather[['temp','rhum','prcp','wdir','wspd','pres']])

In [56]:
final_training = pd.read_csv('Training_Data_Three_Categories.csv')

In [177]:
count = 0
for i in range(0,3000000,30000):
    count += 1
    subset = final_training.loc[i:i+29999]
    subset.to_csv(f'Training_Data_Three_Categories_Part{count}.csv',index=False)

In [178]:
data = [ pd.read_csv(x) for x in os.listdir(os.getcwd()) ]

In [179]:
data = pd.concat(data)

In [180]:
len(data)

3000000

In [181]:
data

,year,julian,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,...,dep_prcp,dep_wdir,dep_wspd,dep_pres,arr_temp,arr_rhum,arr_prcp,arr_wdir,arr_wspd,arr_pres
0,2018,63,DL,968,DL,968,14679,SAN,12478,JFK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,353,AA,1581,AA,1581,13303,MIA,15016,STL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,201,B6,106,B6,106,13930,ORD,12478,JFK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,53,WN,1272,WN,1272,14107,PHX,15016,STL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,226,WN,1182,WN,1182,10140,ABQ,14107,PHX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2019,46,AS,2285,QX,2285,11648,FCA,14747,SEA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29996,2019,266,WN,1965,WN,1965,14570,RNO,12889,LAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29997,2018,182,UA,738,UA,738,11292,DEN,13204,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,2019,57,AA,4785,PT,4785,14100,PHL,10581,BGR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
final_training

,year,julian,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,...,dep_prcp,dep_wdir,dep_wspd,dep_pres,arr_temp,arr_rhum,arr_prcp,arr_wdir,arr_wspd,arr_pres
0,2018,63,DL,968,DL,968,14679,SAN,12478,JFK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,353,AA,1581,AA,1581,13303,MIA,15016,STL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,201,B6,106,B6,106,13930,ORD,12478,JFK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,53,WN,1272,WN,1272,14107,PHX,15016,STL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,226,WN,1182,WN,1182,10140,ABQ,14107,PHX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2018,86,AA,2534,AA,2534,13342,MKE,11298,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999996,2018,205,DL,5106,9E,5106,12953,LGA,10785,BTV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999997,2019,131,UA,4727,AX,4727,11292,DEN,13029,LNK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999998,2019,164,WN,2132,WN,2132,11540,ELP,14107,PHX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import os
import pandas as pd
from sys import argv

weather_data = []
flight_data = pd.read_csv(argv[1])

for folder in ['WeatherData01/', 'WeatherData02/', 'WeatherData03/', 'WeatherData04/', 'WeatherData05/',]:
    os.chdir(folder)
    weather = {}
    for fname in [ x for x in os.listdir(os.getcwd()) if 'hourly' in x ]:
        code = fname.split('_')[0]
        data = pd.read_csv(fname,index_col='time')
        weather[code] = data
    weather_data.append(weather)
    os.chdir('..')
    
def get_dep_weather_data(df):
    for i in range(len(df)):
        if ((i+1) % 1000) == 0:
            print(f'\rProcessing row {(i+1)//1000}k')
        dep_code, dep_dt = df.loc[i].origin, str(df.loc[i].dep_datetime_rounded)
        dep_weather = weather_data[2][dep_code].loc[dep_dt]
        arr_code, arr_dt = df.loc[i].dest, str(df.loc[i].arr_datetime_rounded)
        arr_weather = weather_data[2][arr_code].loc[arr_dt]
        df.loc[i,['dep_temp','dep_rhum','dep_prcp','dep_wdir','dep_wspd','dep_pres']] = list(dep_weather[['temp','rhum','prcp','wdir','wspd','pres']])
        df.loc[i,['arr_temp','arr_rhum','arr_prcp','arr_wdir','arr_wspd','arr_pres']] = list(arr_weather[['temp','rhum','prcp','wdir','wspd','pres']])

get_dep_weather_data(flight_data)

flight_data.to_csv(argv[1].replace('.csv','_with_weather.csv'),index=False)